In [23]:
import pandas as pd 
import numpy as np 

In [3]:
expert_agent = pd.read_csv("data/data_generation_mpc_110_190_6_all.csv")

### Split the data in Features and Targets and Scale using Sklearn

In [5]:
from sklearn import preprocessing

X = expert_agent[["inflow", "height"]]
X_scaler = preprocessing.StandardScaler().fit(X)
X_scaled = X_scaler.transform(X)

y = expert_agent[["speed1_rpm", "speed4_rpm"]]
y_scaler = preprocessing.StandardScaler().fit(y)
y_scaled = y_scaler.transform(y)


### Transform arrays into Torch DataLoader

In [6]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
import torch

seed = torch.Generator().manual_seed(42)

torch_dataset = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), 
                              torch.tensor(y_scaled, dtype=torch.float32))

total_size = len(torch_dataset)
train_size = int(0.7 * total_size)  # 70% of data for training
valid_size = int(0.15 * total_size)  # 15% of data for validation
test_size = total_size - train_size - valid_size  # Remaining 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(torch_dataset, [train_size, valid_size, test_size], generator=seed)

# Create Data Loaders for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Define the NN Skeleton in Pytorch

In [8]:
import pytorch_lightning as pl


import torch
import torch.nn as nn
import torch.optim as optim

# Define the dataset (assuming X_scaled, y_scaled are numpy arrays)
train_dataset = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), 
                              torch.tensor(y_scaled, dtype=torch.float32))

# Define the Neural Network using PyTorch Lightning ==> subclassing
class LitNeuralNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 2)
        self.train_loss = []
        self.val_loss = []

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        train_loss = nn.functional.mse_loss(y_hat, y)
        self.train_loss.append(train_loss.cpu().item())
        self.log('train_loss', train_loss, on_epoch=True, prog_bar=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.functional.mse_loss(y_hat, y)
        self.val_loss.append(val_loss.cpu().item())
        self.log('val_loss', val_loss, on_epoch=True, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        test_loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', test_loss, on_epoch=True)
        return test_loss

    def configure_optimizers(self):
        # Add L2 regularization with weight_decay
        optimizer = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

### Initialize the NN class

In [9]:
model = LitNeuralNet()

### Create Model Early Stopping to avoid Overfitting

In [16]:
import torch.optim as optim
import torch.nn.functional as F

In [17]:
model = LitNeuralNet()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# Training loop
for epoch in range(10):  # Set num_epochs to your desired number
    model.train()
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = F.mse_loss(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for x_batch, y_batch in valid_loader:
            y_pred = model(x_batch)
            val_loss += F.mse_loss(y_pred, y_batch).item()
        val_loss /= len(valid_loader)
        print(f"Epoch {epoch}, Validation Loss: {val_loss}")


Epoch 0, Validation Loss: 0.020886796789484158
Epoch 1, Validation Loss: 0.011192267204845143
Epoch 2, Validation Loss: 0.006354833113390111
Epoch 3, Validation Loss: 0.002702408306480896
Epoch 4, Validation Loss: 0.001847554874914368
Epoch 5, Validation Loss: 0.0013150296139252944
Epoch 6, Validation Loss: 0.0012681312605573405
Epoch 7, Validation Loss: 0.0010614527455125485
Epoch 8, Validation Loss: 0.0010310986912020691
Epoch 9, Validation Loss: 0.0009683445584209215


In [18]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for x_batch, y_batch in test_loader:
        y_pred = model(x_batch)
        test_loss += F.mse_loss(y_pred, y_batch).item()
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss}")

Test Loss: 0.0010382567363839987


#### Save Scaler 

In [78]:
import joblib

# Save X_scaler
joblib.dump(X_scaler, 'data/scalers/X_scaler.pkl')

# Save y_scaler
joblib.dump(y_scaler, 'data/scalers/y_scaler.pkl')


['data/scalers/y_scaler.pkl']

#### Save trained model

In [79]:
torch.save(model.state_dict(), 'data/model/trained_model.pth')